## Data cleaning case study

**Dataset:** 50,000 tweets

**Columns:**
- tweet_id: Unique idenntification number for each tweets
- user_handle: Name of the user who tweeted
- timestamp:Date and time when each tweest was posted
- text: Tweet messages
- likes: Number of likes each tweet got
- retweets: Number of retweets
- sentiment_score: The emotional tone conveyed by each tweet

**Cleaning objectives:**

- Parse timestamp into datetime; handle inconsistent time zones

- Remove bot or spam accounts (e.g. duplicate user_handle with >100 tweets/day)

- Strip URLs/mentions from text; normalize emojis

- Impute missing sentiment_score by mean

Tools: Python (Pandas, regex)

In [ ]:
# Importing necessary libraries
import pandas as pd
import re
import numpy as np
import emoji

In [ ]:
# Uploading dataset with pandas
df = pd.read_csv('/content/drive/MyDrive/Analytical Engineering /Portfolio/Data Cleaning/social_media_sentiment.csv')

In [ ]:
# Reading the first 5 rows of dataset
df.head()

,tweet_id,user_handle,timestamp,text,likes,retweets,sentiment_score
0,1,user_6842,2025-05-11 12:38:30 +05:30,🎉 bad 😢 http://example.com Check fantastic this,8,0,-0.25
1,2,user_3070,2025-05-16T22:18:28+00:00,😢 happy 😢 sad brand! @user this out out love I...,6,4,0.25
2,3,user_9402,2025-06-02T18:37:14+09:00,I Check this out 😢 out I brand! I 😢 I terrible...,3,1,NaN
3,4,user_1027,2025-05-12T10:10:58+05:30,I good out I bad good fantastic @user good Che...,2,0,NaN
4,5,user_9992,2025-05-07 07:58:27 -05:00,sad this 🎉 @user bad happy happy 😢 sad sad,5,4,-0.98


In [ ]:
# Checking the Data types
df.dtypes

,0
tweet_id,int64
user_handle,object
timestamp,object
text,object
likes,int64
retweets,int64
sentiment_score,float64


In [ ]:
# Checking the shape of the data - 50,000 rows and 7 columns
df.shape

(50000, 7)

In [ ]:
# Checking the data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tweet_id         50000 non-null  int64  
 1   user_handle      50000 non-null  object 
 2   timestamp        50000 non-null  object 
 3   text             50000 non-null  object 
 4   likes            50000 non-null  int64  
 5   retweets         50000 non-null  int64  
 6   sentiment_score  24718 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 2.7+ MB


In [ ]:
# Checking the summary statistics of data
df.describe()

,tweet_id,likes,retweets,sentiment_score
count,50000.000000,50000.000000,50000.000000,24718.000000
mean,25000.500000,4.981280,1.991480,-0.000939
std,14433.901067,2.222788,1.408065,0.577608
min,1.000000,0.000000,0.000000,-1.000000
25%,12500.750000,3.000000,1.000000,-0.500000
50%,25000.500000,5.000000,2.000000,-0.010000
75%,37500.250000,6.000000,3.000000,0.500000
max,50000.000000,18.000000,10.000000,1.000000


In [ ]:
# Check data for mising value
df.isnull().sum()

,0
tweet_id,0
user_handle,0
timestamp,0
text,0
likes,0
retweets,0
sentiment_score,25282


In [ ]:
# Replacing missing value for sentiment_score with its mean
df['sentiment_score'] = df['sentiment_score'].fillna(df['sentiment_score'].mean())

In [ ]:
# Re-checking data after filling missing values with mean values
df.isnull().sum()

,0
tweet_id,0
user_handle,0
timestamp,0
text,0
likes,0
retweets,0
sentiment_score,0


#### Parsing timestamp into datetime and handling inconsistent time zones

In [ ]:
# Converting to string and strip
cleaned = df['timestamp'].astype(str).str.strip()

In [ ]:
# Converting  any space between datetime and timezone to 'T'
cleaned = cleaned.str.replace(r"(\d{4}-\d{2}-\d{2}) (\d{2}:\d{2}:\d{2}) ([+\-]\d{2}:\d{2})", r"\1T\2\3", regex=True)

In [ ]:
# Converting  any space between date and time (no timezone)
cleaned = cleaned.str.replace(r"(\d{4}-\d{2}-\d{2}) (\d{2}:\d{2}:\d{2})", r"\1T\2", regex=True)

In [ ]:
# Attempting parsing with timezone normalization
parsed = pd.to_datetime(cleaned, errors='coerce', utc=True)

In [ ]:
# Assigning back to DataFrame
df['timestamp'] = parsed

In [ ]:
# Converting to Africa/Lagos time if parsed
df['timestamp'] = df['timestamp'].dt.tz_convert('Africa/Lagos')

In [ ]:
# Timestamp result
df['timestamp']

,timestamp
0,2025-05-11 08:08:30+01:00
1,2025-05-16 23:18:28+01:00
2,2025-06-02 10:37:14+01:00
3,2025-05-12 05:40:58+01:00
4,2025-05-07 13:58:27+01:00
...,...
49995,2025-04-21 05:30:16+01:00
49996,2025-02-10 14:43:44+01:00
49997,2025-01-20 09:45:13+01:00
49998,2025-04-28 00:21:57+01:00


#### Removing bot or spam accounts (e.g. duplicate user_handle with >100 tweets/day)

In [ ]:
# Counting tweets per user per day
tweet_counts = df.groupby(['user_handle', 'timestamp']).size().reset_index(name='tweet_count')


In [ ]:
tweet_counts

,user_handle,timestamp,tweet_count
0,user_1,2025-02-07 04:39:30+01:00,1
1,user_1,2025-02-26 09:09:45+01:00,1
2,user_1,2025-04-16 19:45:53+01:00,1
3,user_1,2025-05-24 07:04:33+01:00,1
4,user_1,2025-05-28 08:21:40+01:00,1
...,...,...,...
49995,user_9998,2025-05-05 15:08:51+01:00,1
49996,user_9999,2025-03-20 14:36:41+01:00,1
49997,user_9999,2025-03-25 09:53:04+01:00,1
49998,user_9999,2025-05-24 06:41:45+01:00,1


In [ ]:
# Identifying suspicious users (e.g., more than 100 tweets/day)
spam_users = tweet_counts[tweet_counts['tweet_count'] > 100]['user_handle'].unique()

In [ ]:
# No Duplicate tweets per day

spam_users

array([], dtype=object)

In [ ]:
# Filtering out rows from spam users
df_cleaned = df[~df['user_handle'].isin(spam_users)]

In [ ]:
print("Potential spam users detected:", len(spam_users))
print("Original shape:", df.shape)
print("Cleaned shape:", df_cleaned.shape)

Potential spam users detected: 0
Original shape: (50000, 7)
Cleaned shape: (50000, 7)


#### Stripping URLs/mentions from text; normalize emojis

In [ ]:
# Defining cleaning function
def clean_text(text):
    if pd.isna(text):
        return text
    # Remove URLs
    text = re.sub(r'http\S+|www\.\S+', '', text)

    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Normalize emojis (convert to text, e.g., :smile:)
    text = emoji.demojize(text, language='en')

    # Optional: remove colons around emoji text
    text = text.replace(':', ' ')

    # Strip extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
# Applying function to text column
df['clean_text'] = df['text'].apply(clean_text)

In [ ]:
# Previewing cleaned column
print(df[['text', 'clean_text']].head())


                                                text  \
0    🎉 bad 😢 http://example.com Check fantastic this   
1  😢 happy 😢 sad brand! @user this out out love I...   
2  I Check this out 😢 out I brand! I 😢 I terrible...   
3  I good out I bad good fantastic @user good Che...   
4         sad this 🎉 @user bad happy happy 😢 sad sad   

                                          clean_text  
0  party_popper bad crying_face Check fantastic this  
1  crying_face happy crying_face sad brand! this ...  
2  I Check this out crying_face out I brand! I cr...  
3    I good out I bad good fantastic good Check love  
4  sad this party_popper bad happy happy crying_f...  
